In [1]:
import pymol
from pymol import cmd, stored

In [2]:
from numpy import *

In [3]:
import os
def writeIDs(path, refStruct, resID, SASA):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:

       # Create a new directory because it does not exist
       os.makedirs(path)
       print("The new directory is created!")
    
    with open(path + '/' + refStruct + '_SASA.txt', 'w') as wf:
        for i in range(len(resID)):
            wf.write(str(resID[i]) + '\t' + str(SASA[i])) 
            wf.write('\n')

In [4]:
# Tien et al., 2013 (doi:10.1371/journal.pone.0080635)
# and are computed using the Shrake and Rupley algorithm 
# with a probe radius of 1.4 Å
max_sasa_values = {
    'A': 121.0,  # Alanine
    'R': 265.0,  # Arginine
    'N': 187.0,  # Asparagine
    'D': 187.0,  # Aspartic Acid
    'C': 148.0,  # Cysteine
    'Q': 214.0,  # Glutamine
    'E': 214.0,  # Glutamic Acid
    'G': 97.0,   # Glycine
    'H': 216.0,  # Histidine
    'I': 195.0,  # Isoleucine
    'L': 191.0,  # Leucine
    'K': 230.0,  # Lysine
    'M': 203.0,  # Methionine
    'F': 228.0,  # Phenylalanine
    'P': 154.0,  # Proline
    'S': 143.0,  # Serine
    'T': 163.0,  # Threonine
    'W': 264.0,  # Tryptophan
    'Y': 255.0,  # Tyrosine
    'V': 165.0   # Valine
}


# prepare codes

In [5]:
import requests
import os

def download_pdb(pdb_id):
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
    response = requests.get(url)
    
    isExist = os.path.exists('./PDB')
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs('./PDB')
        
    
    if response.status_code == 200:
        with open('./PDB/' + pdb_id + ".pdb", 'wb') as file:
            file.write(response.content)
        print(f"{pdb_id}.pdb downloaded successfully.")
        return True
    else:
        print(f"Failed to retrieve {pdb_id}.pdb")
        return False



In [6]:
def calcSASA_C281(currFrame):
    cmd.reinitialize()

    # Load the structure
    cmd.load(currFrame, "my_structure")

    segment_name = 'PROA'
    # Select the target residue in the specified segment
    selection_query = f"my_structure and resi 281 and segi {segment_name}"
    cmd.select("target_residue", selection_query)

    # Calculate and return the SASA
    sasa_A = cmd.get_area("target_residue")


    segment_name = 'PROB'
    # Select the target residue in the specified segment
    selection_query = f"my_structure and resi 281 and segi {segment_name}"
    cmd.select("target_residue", selection_query)

    # Show the surface for the selection
    # cmd.show("surface", "target_residue")

    # Calculate and return the SASA
    sasa_B = cmd.get_area("target_residue")
    return sasa_A, sasa_B



In [7]:
def calcSASA_D282(currFrame):
    cmd.reinitialize()

    # Load the structure
    cmd.load(currFrame, "my_structure")

    segment_name = 'PROA'
    # Select the target residue in the specified segment
    selection_query = f"my_structure and resi 282 and segi {segment_name}"
    cmd.select("target_residue", selection_query)

    # Calculate and return the SASA
    sasa_A = cmd.get_area("target_residue")


    segment_name = 'PROB'
    # Select the target residue in the specified segment
    selection_query = f"my_structure and resi 282 and segi {segment_name}"
    cmd.select("target_residue", selection_query)

    # Show the surface for the selection
    # cmd.show("surface", "target_residue")

    # Calculate and return the SASA
    sasa_B = cmd.get_area("target_residue")
    return sasa_A, sasa_B


In [8]:
def calcSASA(currFrame, resiIndex):
    cmd.reinitialize()

    # Load the structure
    cmd.load(currFrame, "my_structure")

    segment_name = 'PROA'
    # Select the target residue in the specified segment
    selection_query = f"my_structure and resi {resiIndex} and segi {segment_name}"
    cmd.select("target_residue", selection_query)

    # Calculate and return the SASA
    sasa_A = cmd.get_area("target_residue")


    segment_name = 'PROB'
    # Select the target residue in the specified segment
    selection_query = f"my_structure and resi {resiIndex} and segi {segment_name}"
    cmd.select("target_residue", selection_query)

    # Show the surface for the selection
    # cmd.show("surface", "target_residue")

    # Calculate and return the SASA
    sasa_B = cmd.get_area("target_residue")
    return sasa_A, sasa_B


# Prepare system

In [9]:
currFrame = './V135A/Run1_frame/frame_0862.pdb'

In [10]:
segment_name = 'PROA'

# calcSASA

In [11]:
# all distances
mutations = ['WT','N272H', 'G188Q', 'L155W', 'V135A']
Runs = ['Run1', 'Run2', 'Run3', 'Run4', 'Run5']

frames = ["{:04}".format(i) for i in range(1, 1001)]

In [41]:
# all distances

Mercury_SASA = {}
for currMutation in mutations:
    if currMutation not in Mercury_SASA:
        Mercury_SASA[currMutation] = {}
    print ('Start evaluating ' + currMutation)
    for currRuns in Runs:
        print ('Start evaluating ' + currRuns)
        currRun = currRuns + '_frame'
        if currRuns not in Mercury_SASA[currMutation].keys():
            Mercury_SASA[currMutation][currRuns] = {}
        
        for frame in [f"frame_{i:04d}.pdb" for i in range(1, 1001)]:
        
            currDirectory = './' + currMutation + '/' + currRun + '/' + frame

            if 'C281' not in Mercury_SASA[currMutation][currRuns].keys():
                Mercury_SASA[currMutation][currRuns]['C281'] = {'1':[], '2': []}
            if 'D282' not in Mercury_SASA[currMutation][currRuns].keys():
                Mercury_SASA[currMutation][currRuns]['D282'] = {'1':[], '2': []}
            
            area_A, area_B = calcSASA_C281(currDirectory)
            # area_A, area_B = calcSASA_BINDING(currDirectory)

            Mercury_SASA[currMutation][currRuns]['C281']['1'].append(area_A)
            Mercury_SASA[currMutation][currRuns]['C281']['2'].append(area_B)
            print (currDirectory)
            
            area_A, area_B = calcSASA_D282(currDirectory)
            Mercury_SASA[currMutation][currRuns]['D282']['1'].append(area_A)
            Mercury_SASA[currMutation][currRuns]['D282']['2'].append(area_B)

Start evaluating WT
Start evaluating Run1
./WT/Run1_frame/frame_0001.pdb
./WT/Run1_frame/frame_0002.pdb
./WT/Run1_frame/frame_0003.pdb
./WT/Run1_frame/frame_0004.pdb
./WT/Run1_frame/frame_0005.pdb
./WT/Run1_frame/frame_0006.pdb
./WT/Run1_frame/frame_0007.pdb
./WT/Run1_frame/frame_0008.pdb
./WT/Run1_frame/frame_0009.pdb
./WT/Run1_frame/frame_0010.pdb
./WT/Run1_frame/frame_0011.pdb
./WT/Run1_frame/frame_0012.pdb
./WT/Run1_frame/frame_0013.pdb
./WT/Run1_frame/frame_0014.pdb
./WT/Run1_frame/frame_0015.pdb
./WT/Run1_frame/frame_0016.pdb
./WT/Run1_frame/frame_0017.pdb
./WT/Run1_frame/frame_0018.pdb
./WT/Run1_frame/frame_0019.pdb
./WT/Run1_frame/frame_0020.pdb
./WT/Run1_frame/frame_0021.pdb
./WT/Run1_frame/frame_0022.pdb
./WT/Run1_frame/frame_0023.pdb
./WT/Run1_frame/frame_0024.pdb
./WT/Run1_frame/frame_0025.pdb
./WT/Run1_frame/frame_0026.pdb
./WT/Run1_frame/frame_0027.pdb
./WT/Run1_frame/frame_0028.pdb
./WT/Run1_frame/frame_0029.pdb
./WT/Run1_frame/frame_0030.pdb
./WT/Run1_frame/frame_0031.p

./WT/Run1_frame/frame_0264.pdb
./WT/Run1_frame/frame_0265.pdb
./WT/Run1_frame/frame_0266.pdb
./WT/Run1_frame/frame_0267.pdb
./WT/Run1_frame/frame_0268.pdb
./WT/Run1_frame/frame_0269.pdb
./WT/Run1_frame/frame_0270.pdb
./WT/Run1_frame/frame_0271.pdb
./WT/Run1_frame/frame_0272.pdb
./WT/Run1_frame/frame_0273.pdb
./WT/Run1_frame/frame_0274.pdb
./WT/Run1_frame/frame_0275.pdb
./WT/Run1_frame/frame_0276.pdb
./WT/Run1_frame/frame_0277.pdb
./WT/Run1_frame/frame_0278.pdb
./WT/Run1_frame/frame_0279.pdb
./WT/Run1_frame/frame_0280.pdb
./WT/Run1_frame/frame_0281.pdb
./WT/Run1_frame/frame_0282.pdb
./WT/Run1_frame/frame_0283.pdb
./WT/Run1_frame/frame_0284.pdb
./WT/Run1_frame/frame_0285.pdb
./WT/Run1_frame/frame_0286.pdb
./WT/Run1_frame/frame_0287.pdb
./WT/Run1_frame/frame_0288.pdb
./WT/Run1_frame/frame_0289.pdb
./WT/Run1_frame/frame_0290.pdb
./WT/Run1_frame/frame_0291.pdb
./WT/Run1_frame/frame_0292.pdb
./WT/Run1_frame/frame_0293.pdb
./WT/Run1_frame/frame_0294.pdb
./WT/Run1_frame/frame_0295.pdb
./WT/Run

./WT/Run1_frame/frame_0529.pdb
./WT/Run1_frame/frame_0530.pdb
./WT/Run1_frame/frame_0531.pdb
./WT/Run1_frame/frame_0532.pdb
./WT/Run1_frame/frame_0533.pdb
./WT/Run1_frame/frame_0534.pdb
./WT/Run1_frame/frame_0535.pdb
./WT/Run1_frame/frame_0536.pdb
./WT/Run1_frame/frame_0537.pdb
./WT/Run1_frame/frame_0538.pdb
./WT/Run1_frame/frame_0539.pdb
./WT/Run1_frame/frame_0540.pdb
./WT/Run1_frame/frame_0541.pdb
./WT/Run1_frame/frame_0542.pdb
./WT/Run1_frame/frame_0543.pdb
./WT/Run1_frame/frame_0544.pdb
./WT/Run1_frame/frame_0545.pdb
./WT/Run1_frame/frame_0546.pdb
./WT/Run1_frame/frame_0547.pdb
./WT/Run1_frame/frame_0548.pdb
./WT/Run1_frame/frame_0549.pdb
./WT/Run1_frame/frame_0550.pdb
./WT/Run1_frame/frame_0551.pdb
./WT/Run1_frame/frame_0552.pdb
./WT/Run1_frame/frame_0553.pdb
./WT/Run1_frame/frame_0554.pdb
./WT/Run1_frame/frame_0555.pdb
./WT/Run1_frame/frame_0556.pdb
./WT/Run1_frame/frame_0557.pdb
./WT/Run1_frame/frame_0558.pdb
./WT/Run1_frame/frame_0559.pdb
./WT/Run1_frame/frame_0560.pdb
./WT/Run

./WT/Run1_frame/frame_0794.pdb
./WT/Run1_frame/frame_0795.pdb
./WT/Run1_frame/frame_0796.pdb
./WT/Run1_frame/frame_0797.pdb
./WT/Run1_frame/frame_0798.pdb
./WT/Run1_frame/frame_0799.pdb
./WT/Run1_frame/frame_0800.pdb
./WT/Run1_frame/frame_0801.pdb
./WT/Run1_frame/frame_0802.pdb
./WT/Run1_frame/frame_0803.pdb
./WT/Run1_frame/frame_0804.pdb
./WT/Run1_frame/frame_0805.pdb
./WT/Run1_frame/frame_0806.pdb
./WT/Run1_frame/frame_0807.pdb
./WT/Run1_frame/frame_0808.pdb
./WT/Run1_frame/frame_0809.pdb
./WT/Run1_frame/frame_0810.pdb
./WT/Run1_frame/frame_0811.pdb
./WT/Run1_frame/frame_0812.pdb
./WT/Run1_frame/frame_0813.pdb
./WT/Run1_frame/frame_0814.pdb
./WT/Run1_frame/frame_0815.pdb
./WT/Run1_frame/frame_0816.pdb
./WT/Run1_frame/frame_0817.pdb
./WT/Run1_frame/frame_0818.pdb
./WT/Run1_frame/frame_0819.pdb
./WT/Run1_frame/frame_0820.pdb
./WT/Run1_frame/frame_0821.pdb
./WT/Run1_frame/frame_0822.pdb
./WT/Run1_frame/frame_0823.pdb
./WT/Run1_frame/frame_0824.pdb
./WT/Run1_frame/frame_0825.pdb
./WT/Run

./WT/Run2_frame/frame_0058.pdb
./WT/Run2_frame/frame_0059.pdb
./WT/Run2_frame/frame_0060.pdb
./WT/Run2_frame/frame_0061.pdb
./WT/Run2_frame/frame_0062.pdb
./WT/Run2_frame/frame_0063.pdb
./WT/Run2_frame/frame_0064.pdb
./WT/Run2_frame/frame_0065.pdb
./WT/Run2_frame/frame_0066.pdb
./WT/Run2_frame/frame_0067.pdb
./WT/Run2_frame/frame_0068.pdb
./WT/Run2_frame/frame_0069.pdb
./WT/Run2_frame/frame_0070.pdb
./WT/Run2_frame/frame_0071.pdb
./WT/Run2_frame/frame_0072.pdb
./WT/Run2_frame/frame_0073.pdb
./WT/Run2_frame/frame_0074.pdb
./WT/Run2_frame/frame_0075.pdb
./WT/Run2_frame/frame_0076.pdb
./WT/Run2_frame/frame_0077.pdb
./WT/Run2_frame/frame_0078.pdb
./WT/Run2_frame/frame_0079.pdb
./WT/Run2_frame/frame_0080.pdb
./WT/Run2_frame/frame_0081.pdb
./WT/Run2_frame/frame_0082.pdb
./WT/Run2_frame/frame_0083.pdb
./WT/Run2_frame/frame_0084.pdb
./WT/Run2_frame/frame_0085.pdb
./WT/Run2_frame/frame_0086.pdb
./WT/Run2_frame/frame_0087.pdb
./WT/Run2_frame/frame_0088.pdb
./WT/Run2_frame/frame_0089.pdb
./WT/Run

...

./V135A/Run4_frame/frame_0882.pdb
./V135A/Run4_frame/frame_0883.pdb
./V135A/Run4_frame/frame_0884.pdb
./V135A/Run4_frame/frame_0885.pdb
./V135A/Run4_frame/frame_0886.pdb
./V135A/Run4_frame/frame_0887.pdb
./V135A/Run4_frame/frame_0888.pdb
./V135A/Run4_frame/frame_0889.pdb
./V135A/Run4_frame/frame_0890.pdb
./V135A/Run4_frame/frame_0891.pdb
./V135A/Run4_frame/frame_0892.pdb
./V135A/Run4_frame/frame_0893.pdb
./V135A/Run4_frame/frame_0894.pdb
./V135A/Run4_frame/frame_0895.pdb
./V135A/Run4_frame/frame_0896.pdb
./V135A/Run4_frame/frame_0897.pdb
./V135A/Run4_frame/frame_0898.pdb
./V135A/Run4_frame/frame_0899.pdb
./V135A/Run4_frame/frame_0900.pdb
./V135A/Run4_frame/frame_0901.pdb
./V135A/Run4_frame/frame_0902.pdb
./V135A/Run4_frame/frame_0903.pdb
./V135A/Run4_frame/frame_0904.pdb
./V135A/Run4_frame/frame_0905.pdb
./V135A/Run4_frame/frame_0906.pdb
./V135A/Run4_frame/frame_0907.pdb
./V135A/Run4_frame/frame_0908.pdb
./V135A/Run4_frame/frame_0909.pdb
./V135A/Run4_frame/frame_0910.pdb
./V135A/Run4_f

./V135A/Run5_frame/frame_0123.pdb
./V135A/Run5_frame/frame_0124.pdb
./V135A/Run5_frame/frame_0125.pdb
./V135A/Run5_frame/frame_0126.pdb
./V135A/Run5_frame/frame_0127.pdb
./V135A/Run5_frame/frame_0128.pdb
./V135A/Run5_frame/frame_0129.pdb
./V135A/Run5_frame/frame_0130.pdb
./V135A/Run5_frame/frame_0131.pdb
./V135A/Run5_frame/frame_0132.pdb
./V135A/Run5_frame/frame_0133.pdb
./V135A/Run5_frame/frame_0134.pdb
./V135A/Run5_frame/frame_0135.pdb
./V135A/Run5_frame/frame_0136.pdb
./V135A/Run5_frame/frame_0137.pdb
./V135A/Run5_frame/frame_0138.pdb
./V135A/Run5_frame/frame_0139.pdb
./V135A/Run5_frame/frame_0140.pdb
./V135A/Run5_frame/frame_0141.pdb
./V135A/Run5_frame/frame_0142.pdb
./V135A/Run5_frame/frame_0143.pdb
./V135A/Run5_frame/frame_0144.pdb
./V135A/Run5_frame/frame_0145.pdb
./V135A/Run5_frame/frame_0146.pdb
./V135A/Run5_frame/frame_0147.pdb
./V135A/Run5_frame/frame_0148.pdb
./V135A/Run5_frame/frame_0149.pdb
./V135A/Run5_frame/frame_0150.pdb
./V135A/Run5_frame/frame_0151.pdb
./V135A/Run5_f

./V135A/Run5_frame/frame_0364.pdb
./V135A/Run5_frame/frame_0365.pdb
./V135A/Run5_frame/frame_0366.pdb
./V135A/Run5_frame/frame_0367.pdb
./V135A/Run5_frame/frame_0368.pdb
./V135A/Run5_frame/frame_0369.pdb
./V135A/Run5_frame/frame_0370.pdb
./V135A/Run5_frame/frame_0371.pdb
./V135A/Run5_frame/frame_0372.pdb
./V135A/Run5_frame/frame_0373.pdb
./V135A/Run5_frame/frame_0374.pdb
./V135A/Run5_frame/frame_0375.pdb
./V135A/Run5_frame/frame_0376.pdb
./V135A/Run5_frame/frame_0377.pdb
./V135A/Run5_frame/frame_0378.pdb
./V135A/Run5_frame/frame_0379.pdb
./V135A/Run5_frame/frame_0380.pdb
./V135A/Run5_frame/frame_0381.pdb
./V135A/Run5_frame/frame_0382.pdb
./V135A/Run5_frame/frame_0383.pdb
./V135A/Run5_frame/frame_0384.pdb
./V135A/Run5_frame/frame_0385.pdb
./V135A/Run5_frame/frame_0386.pdb
./V135A/Run5_frame/frame_0387.pdb
./V135A/Run5_frame/frame_0388.pdb
./V135A/Run5_frame/frame_0389.pdb
./V135A/Run5_frame/frame_0390.pdb
./V135A/Run5_frame/frame_0391.pdb
./V135A/Run5_frame/frame_0392.pdb
./V135A/Run5_f

./V135A/Run5_frame/frame_0605.pdb
./V135A/Run5_frame/frame_0606.pdb
./V135A/Run5_frame/frame_0607.pdb
./V135A/Run5_frame/frame_0608.pdb
./V135A/Run5_frame/frame_0609.pdb
./V135A/Run5_frame/frame_0610.pdb
./V135A/Run5_frame/frame_0611.pdb
./V135A/Run5_frame/frame_0612.pdb
./V135A/Run5_frame/frame_0613.pdb
./V135A/Run5_frame/frame_0614.pdb
./V135A/Run5_frame/frame_0615.pdb
./V135A/Run5_frame/frame_0616.pdb
./V135A/Run5_frame/frame_0617.pdb
./V135A/Run5_frame/frame_0618.pdb
./V135A/Run5_frame/frame_0619.pdb
./V135A/Run5_frame/frame_0620.pdb
./V135A/Run5_frame/frame_0621.pdb
./V135A/Run5_frame/frame_0622.pdb
./V135A/Run5_frame/frame_0623.pdb
./V135A/Run5_frame/frame_0624.pdb
./V135A/Run5_frame/frame_0625.pdb
./V135A/Run5_frame/frame_0626.pdb
./V135A/Run5_frame/frame_0627.pdb
./V135A/Run5_frame/frame_0628.pdb
./V135A/Run5_frame/frame_0629.pdb
./V135A/Run5_frame/frame_0630.pdb
./V135A/Run5_frame/frame_0631.pdb
./V135A/Run5_frame/frame_0632.pdb
./V135A/Run5_frame/frame_0633.pdb
./V135A/Run5_f

./V135A/Run5_frame/frame_0846.pdb
./V135A/Run5_frame/frame_0847.pdb
./V135A/Run5_frame/frame_0848.pdb
./V135A/Run5_frame/frame_0849.pdb
./V135A/Run5_frame/frame_0850.pdb
./V135A/Run5_frame/frame_0851.pdb
./V135A/Run5_frame/frame_0852.pdb
./V135A/Run5_frame/frame_0853.pdb
./V135A/Run5_frame/frame_0854.pdb
./V135A/Run5_frame/frame_0855.pdb
./V135A/Run5_frame/frame_0856.pdb
./V135A/Run5_frame/frame_0857.pdb
./V135A/Run5_frame/frame_0858.pdb
./V135A/Run5_frame/frame_0859.pdb
./V135A/Run5_frame/frame_0860.pdb
./V135A/Run5_frame/frame_0861.pdb
./V135A/Run5_frame/frame_0862.pdb
./V135A/Run5_frame/frame_0863.pdb
./V135A/Run5_frame/frame_0864.pdb
./V135A/Run5_frame/frame_0865.pdb
./V135A/Run5_frame/frame_0866.pdb
./V135A/Run5_frame/frame_0867.pdb
./V135A/Run5_frame/frame_0868.pdb
./V135A/Run5_frame/frame_0869.pdb
./V135A/Run5_frame/frame_0870.pdb
./V135A/Run5_frame/frame_0871.pdb
./V135A/Run5_frame/frame_0872.pdb
./V135A/Run5_frame/frame_0873.pdb
./V135A/Run5_frame/frame_0874.pdb
./V135A/Run5_f

In [57]:
Mercury_SASA.keys()

dict_keys(['WT', 'V135A'])

In [58]:
Mercury_SASA['V135A']['Run1']['C281']['1'] == Mercury_SASA['V135A']['Run1']['C281']['2']

False

In [65]:
# ['Volumes', 'TotalSASA', 'PolarSASA', 'Hydrophobic Density', 'Charge']
currFeature = 'D282'
cutoff = 0
chain = '2'

data_distribution = []

for mutate in Mercury_SASA.keys():
    list1 = Mercury_SASA[mutate]['Run1'][currFeature][chain][cutoff:]
    list2 = Mercury_SASA[mutate]['Run2'][currFeature][chain][cutoff:]
    list3 = Mercury_SASA[mutate]['Run3'][currFeature][chain][cutoff:]
    list4 = Mercury_SASA[mutate]['Run4'][currFeature][chain][cutoff:]
    list5 = Mercury_SASA[mutate]['Run5'][currFeature][chain][cutoff:]

    
    data_distribution.append(list1 + list2 + list3 + list4 + list5)
    

In [71]:
data = Mercury_SASA

In [84]:
# Prepare DataFrame columns and data
columns = []
data_lists = []

# Process data
for mutation in data:
    for run in data[mutation]:
        for dist_key, chains in data[mutation][run].items():
            for chain_id, values in chains.items():
                col_name = f"{mutation}_{run}_{dist_key}_{chain_id}"
                columns.append(col_name)
                data_lists.append(values)  # Append the whole list, not extend


In [100]:
titles = columns

data = array(data_lists).T.tolist()

In [99]:
len(newList)

1000

In [89]:
import csv

In [101]:
# File path to write
file_path = 'mercury_SASA_all.txt'

# Writing to a file
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')  # Use tab as delimiter
    writer.writerow(titles)  # Write the title row
    writer.writerows(data)  # Write the data rows

In [67]:
std(data_distribution[0])

3.2801321560906795

In [68]:
mean(data_distribution[1])

97.16899195098877

In [14]:
# all distances

Mercury_SASA_ext = {}
for currMutation in mutations:
    if currMutation not in Mercury_SASA_ext:
        Mercury_SASA_ext[currMutation] = {}
    print ('Start evaluating ' + currMutation)
    for currRuns in Runs:
        print ('Start evaluating ' + currRuns)
        currRun = currRuns + '_frame'
        if currRuns not in Mercury_SASA_ext[currMutation].keys():
            Mercury_SASA_ext[currMutation][currRuns] = {}
        
        for frame in [f"frame_{i:04d}.pdb" for i in range(1, 1001)]:
        
            currDirectory = './' + currMutation + '/' + currRun + '/' + frame

            if 'C164' not in Mercury_SASA_ext[currMutation][currRuns].keys():
                Mercury_SASA_ext[currMutation][currRuns]['C164'] = {'1':[], '2': []}
            if 'C192' not in Mercury_SASA_ext[currMutation][currRuns].keys():
                Mercury_SASA_ext[currMutation][currRuns]['C192'] = {'1':[], '2': []}
            if 'C239' not in Mercury_SASA_ext[currMutation][currRuns].keys():
                Mercury_SASA_ext[currMutation][currRuns]['C239'] = {'1':[], '2': []}
            if 'C357' not in Mercury_SASA_ext[currMutation][currRuns].keys():
                Mercury_SASA_ext[currMutation][currRuns]['C357'] = {'1':[], '2': []}
            if 'C281' not in Mercury_SASA_ext[currMutation][currRuns].keys():
                Mercury_SASA_ext[currMutation][currRuns]['C281'] = {'1':[], '2': []}
            if 'D282' not in Mercury_SASA_ext[currMutation][currRuns].keys():
                Mercury_SASA_ext[currMutation][currRuns]['D282'] = {'1':[], '2': []}
            
            area_A, area_B = calcSASA_C281(currDirectory)
            # area_A, area_B = calcSASA_BINDING(currDirectory)

            Mercury_SASA_ext[currMutation][currRuns]['C281']['1'].append(area_A)
            Mercury_SASA_ext[currMutation][currRuns]['C281']['2'].append(area_B)
            # print (currDirectory)
            
            area_A, area_B = calcSASA_D282(currDirectory)
            Mercury_SASA_ext[currMutation][currRuns]['D282']['1'].append(area_A)
            Mercury_SASA_ext[currMutation][currRuns]['D282']['2'].append(area_B)
            
            area_A, area_B = calcSASA(currDirectory, '164')
            # area_A, area_B = calcSASA_BINDING(currDirectory)

            Mercury_SASA_ext[currMutation][currRuns]['C164']['1'].append(area_A)
            Mercury_SASA_ext[currMutation][currRuns]['C164']['2'].append(area_B)
            print (currDirectory)
            
            area_A, area_B = calcSASA(currDirectory, '192')
            Mercury_SASA_ext[currMutation][currRuns]['C192']['1'].append(area_A)
            Mercury_SASA_ext[currMutation][currRuns]['C192']['2'].append(area_B)
            
            area_A, area_B = calcSASA(currDirectory, '239')
            Mercury_SASA_ext[currMutation][currRuns]['C239']['1'].append(area_A)
            Mercury_SASA_ext[currMutation][currRuns]['C239']['2'].append(area_B)
            
            
            area_A, area_B = calcSASA(currDirectory, '357')
            Mercury_SASA_ext[currMutation][currRuns]['C357']['1'].append(area_A)
            Mercury_SASA_ext[currMutation][currRuns]['C357']['2'].append(area_B)


Start evaluating WT
Start evaluating Run1
./WT/Run1_frame/frame_0001.pdb
./WT/Run1_frame/frame_0002.pdb
./WT/Run1_frame/frame_0003.pdb
./WT/Run1_frame/frame_0004.pdb
./WT/Run1_frame/frame_0005.pdb
./WT/Run1_frame/frame_0006.pdb
./WT/Run1_frame/frame_0007.pdb
./WT/Run1_frame/frame_0008.pdb
./WT/Run1_frame/frame_0009.pdb
./WT/Run1_frame/frame_0010.pdb
./WT/Run1_frame/frame_0011.pdb
./WT/Run1_frame/frame_0012.pdb
./WT/Run1_frame/frame_0013.pdb
./WT/Run1_frame/frame_0014.pdb
./WT/Run1_frame/frame_0015.pdb
./WT/Run1_frame/frame_0016.pdb
./WT/Run1_frame/frame_0017.pdb
./WT/Run1_frame/frame_0018.pdb
./WT/Run1_frame/frame_0019.pdb
./WT/Run1_frame/frame_0020.pdb
./WT/Run1_frame/frame_0021.pdb
./WT/Run1_frame/frame_0022.pdb
./WT/Run1_frame/frame_0023.pdb
./WT/Run1_frame/frame_0024.pdb
./WT/Run1_frame/frame_0025.pdb
./WT/Run1_frame/frame_0026.pdb
./WT/Run1_frame/frame_0027.pdb
./WT/Run1_frame/frame_0028.pdb
./WT/Run1_frame/frame_0029.pdb
./WT/Run1_frame/frame_0030.pdb
./WT/Run1_frame/frame_0031.p

./WT/Run1_frame/frame_0264.pdb
./WT/Run1_frame/frame_0265.pdb
./WT/Run1_frame/frame_0266.pdb
./WT/Run1_frame/frame_0267.pdb
./WT/Run1_frame/frame_0268.pdb
./WT/Run1_frame/frame_0269.pdb
./WT/Run1_frame/frame_0270.pdb
./WT/Run1_frame/frame_0271.pdb
./WT/Run1_frame/frame_0272.pdb
./WT/Run1_frame/frame_0273.pdb
./WT/Run1_frame/frame_0274.pdb
./WT/Run1_frame/frame_0275.pdb
./WT/Run1_frame/frame_0276.pdb
./WT/Run1_frame/frame_0277.pdb
./WT/Run1_frame/frame_0278.pdb
./WT/Run1_frame/frame_0279.pdb
./WT/Run1_frame/frame_0280.pdb
./WT/Run1_frame/frame_0281.pdb
./WT/Run1_frame/frame_0282.pdb
./WT/Run1_frame/frame_0283.pdb
./WT/Run1_frame/frame_0284.pdb
./WT/Run1_frame/frame_0285.pdb
./WT/Run1_frame/frame_0286.pdb
./WT/Run1_frame/frame_0287.pdb
./WT/Run1_frame/frame_0288.pdb
./WT/Run1_frame/frame_0289.pdb
./WT/Run1_frame/frame_0290.pdb
./WT/Run1_frame/frame_0291.pdb
./WT/Run1_frame/frame_0292.pdb
./WT/Run1_frame/frame_0293.pdb
./WT/Run1_frame/frame_0294.pdb
./WT/Run1_frame/frame_0295.pdb
./WT/Run

./WT/Run1_frame/frame_0529.pdb
./WT/Run1_frame/frame_0530.pdb
./WT/Run1_frame/frame_0531.pdb
./WT/Run1_frame/frame_0532.pdb
./WT/Run1_frame/frame_0533.pdb
./WT/Run1_frame/frame_0534.pdb
./WT/Run1_frame/frame_0535.pdb
./WT/Run1_frame/frame_0536.pdb
./WT/Run1_frame/frame_0537.pdb
./WT/Run1_frame/frame_0538.pdb
./WT/Run1_frame/frame_0539.pdb
./WT/Run1_frame/frame_0540.pdb
./WT/Run1_frame/frame_0541.pdb
./WT/Run1_frame/frame_0542.pdb
./WT/Run1_frame/frame_0543.pdb
./WT/Run1_frame/frame_0544.pdb
./WT/Run1_frame/frame_0545.pdb
./WT/Run1_frame/frame_0546.pdb
./WT/Run1_frame/frame_0547.pdb
./WT/Run1_frame/frame_0548.pdb
./WT/Run1_frame/frame_0549.pdb
./WT/Run1_frame/frame_0550.pdb
./WT/Run1_frame/frame_0551.pdb
./WT/Run1_frame/frame_0552.pdb
./WT/Run1_frame/frame_0553.pdb
./WT/Run1_frame/frame_0554.pdb
./WT/Run1_frame/frame_0555.pdb
./WT/Run1_frame/frame_0556.pdb
./WT/Run1_frame/frame_0557.pdb
./WT/Run1_frame/frame_0558.pdb
./WT/Run1_frame/frame_0559.pdb
./WT/Run1_frame/frame_0560.pdb
./WT/Run

./WT/Run1_frame/frame_0794.pdb
./WT/Run1_frame/frame_0795.pdb
./WT/Run1_frame/frame_0796.pdb
./WT/Run1_frame/frame_0797.pdb
./WT/Run1_frame/frame_0798.pdb
./WT/Run1_frame/frame_0799.pdb
./WT/Run1_frame/frame_0800.pdb
./WT/Run1_frame/frame_0801.pdb
./WT/Run1_frame/frame_0802.pdb
./WT/Run1_frame/frame_0803.pdb
./WT/Run1_frame/frame_0804.pdb
./WT/Run1_frame/frame_0805.pdb
./WT/Run1_frame/frame_0806.pdb
./WT/Run1_frame/frame_0807.pdb
./WT/Run1_frame/frame_0808.pdb
./WT/Run1_frame/frame_0809.pdb
./WT/Run1_frame/frame_0810.pdb
./WT/Run1_frame/frame_0811.pdb
./WT/Run1_frame/frame_0812.pdb
./WT/Run1_frame/frame_0813.pdb
./WT/Run1_frame/frame_0814.pdb
./WT/Run1_frame/frame_0815.pdb
./WT/Run1_frame/frame_0816.pdb
./WT/Run1_frame/frame_0817.pdb
./WT/Run1_frame/frame_0818.pdb
./WT/Run1_frame/frame_0819.pdb
./WT/Run1_frame/frame_0820.pdb
./WT/Run1_frame/frame_0821.pdb
./WT/Run1_frame/frame_0822.pdb
./WT/Run1_frame/frame_0823.pdb
./WT/Run1_frame/frame_0824.pdb
./WT/Run1_frame/frame_0825.pdb
./WT/Run

./WT/Run2_frame/frame_0058.pdb
./WT/Run2_frame/frame_0059.pdb
./WT/Run2_frame/frame_0060.pdb
./WT/Run2_frame/frame_0061.pdb
./WT/Run2_frame/frame_0062.pdb
./WT/Run2_frame/frame_0063.pdb
./WT/Run2_frame/frame_0064.pdb
./WT/Run2_frame/frame_0065.pdb
./WT/Run2_frame/frame_0066.pdb
./WT/Run2_frame/frame_0067.pdb
./WT/Run2_frame/frame_0068.pdb
./WT/Run2_frame/frame_0069.pdb
./WT/Run2_frame/frame_0070.pdb
./WT/Run2_frame/frame_0071.pdb
./WT/Run2_frame/frame_0072.pdb
./WT/Run2_frame/frame_0073.pdb
./WT/Run2_frame/frame_0074.pdb
./WT/Run2_frame/frame_0075.pdb
./WT/Run2_frame/frame_0076.pdb
./WT/Run2_frame/frame_0077.pdb
./WT/Run2_frame/frame_0078.pdb
./WT/Run2_frame/frame_0079.pdb
./WT/Run2_frame/frame_0080.pdb
./WT/Run2_frame/frame_0081.pdb
./WT/Run2_frame/frame_0082.pdb
./WT/Run2_frame/frame_0083.pdb
./WT/Run2_frame/frame_0084.pdb
./WT/Run2_frame/frame_0085.pdb
./WT/Run2_frame/frame_0086.pdb
./WT/Run2_frame/frame_0087.pdb
./WT/Run2_frame/frame_0088.pdb
./WT/Run2_frame/frame_0089.pdb
./WT/Run

...

./V135A/Run4_frame/frame_0821.pdb
./V135A/Run4_frame/frame_0822.pdb
./V135A/Run4_frame/frame_0823.pdb
./V135A/Run4_frame/frame_0824.pdb
./V135A/Run4_frame/frame_0825.pdb
./V135A/Run4_frame/frame_0826.pdb
./V135A/Run4_frame/frame_0827.pdb
./V135A/Run4_frame/frame_0828.pdb
./V135A/Run4_frame/frame_0829.pdb
./V135A/Run4_frame/frame_0830.pdb
./V135A/Run4_frame/frame_0831.pdb
./V135A/Run4_frame/frame_0832.pdb
./V135A/Run4_frame/frame_0833.pdb
./V135A/Run4_frame/frame_0834.pdb
./V135A/Run4_frame/frame_0835.pdb
./V135A/Run4_frame/frame_0836.pdb
./V135A/Run4_frame/frame_0837.pdb
./V135A/Run4_frame/frame_0838.pdb
./V135A/Run4_frame/frame_0839.pdb
./V135A/Run4_frame/frame_0840.pdb
./V135A/Run4_frame/frame_0841.pdb
./V135A/Run4_frame/frame_0842.pdb
./V135A/Run4_frame/frame_0843.pdb
./V135A/Run4_frame/frame_0844.pdb
./V135A/Run4_frame/frame_0845.pdb
./V135A/Run4_frame/frame_0846.pdb
./V135A/Run4_frame/frame_0847.pdb
./V135A/Run4_frame/frame_0848.pdb
./V135A/Run4_frame/frame_0849.pdb
./V135A/Run4_f

./V135A/Run5_frame/frame_0062.pdb
./V135A/Run5_frame/frame_0063.pdb
./V135A/Run5_frame/frame_0064.pdb
./V135A/Run5_frame/frame_0065.pdb
./V135A/Run5_frame/frame_0066.pdb
./V135A/Run5_frame/frame_0067.pdb
./V135A/Run5_frame/frame_0068.pdb
./V135A/Run5_frame/frame_0069.pdb
./V135A/Run5_frame/frame_0070.pdb
./V135A/Run5_frame/frame_0071.pdb
./V135A/Run5_frame/frame_0072.pdb
./V135A/Run5_frame/frame_0073.pdb
./V135A/Run5_frame/frame_0074.pdb
./V135A/Run5_frame/frame_0075.pdb
./V135A/Run5_frame/frame_0076.pdb
./V135A/Run5_frame/frame_0077.pdb
./V135A/Run5_frame/frame_0078.pdb
./V135A/Run5_frame/frame_0079.pdb
./V135A/Run5_frame/frame_0080.pdb
./V135A/Run5_frame/frame_0081.pdb
./V135A/Run5_frame/frame_0082.pdb
./V135A/Run5_frame/frame_0083.pdb
./V135A/Run5_frame/frame_0084.pdb
./V135A/Run5_frame/frame_0085.pdb
./V135A/Run5_frame/frame_0086.pdb
./V135A/Run5_frame/frame_0087.pdb
./V135A/Run5_frame/frame_0088.pdb
./V135A/Run5_frame/frame_0089.pdb
./V135A/Run5_frame/frame_0090.pdb
./V135A/Run5_f

./V135A/Run5_frame/frame_0303.pdb
./V135A/Run5_frame/frame_0304.pdb
./V135A/Run5_frame/frame_0305.pdb
./V135A/Run5_frame/frame_0306.pdb
./V135A/Run5_frame/frame_0307.pdb
./V135A/Run5_frame/frame_0308.pdb
./V135A/Run5_frame/frame_0309.pdb
./V135A/Run5_frame/frame_0310.pdb
./V135A/Run5_frame/frame_0311.pdb
./V135A/Run5_frame/frame_0312.pdb
./V135A/Run5_frame/frame_0313.pdb
./V135A/Run5_frame/frame_0314.pdb
./V135A/Run5_frame/frame_0315.pdb
./V135A/Run5_frame/frame_0316.pdb
./V135A/Run5_frame/frame_0317.pdb
./V135A/Run5_frame/frame_0318.pdb
./V135A/Run5_frame/frame_0319.pdb
./V135A/Run5_frame/frame_0320.pdb
./V135A/Run5_frame/frame_0321.pdb
./V135A/Run5_frame/frame_0322.pdb
./V135A/Run5_frame/frame_0323.pdb
./V135A/Run5_frame/frame_0324.pdb
./V135A/Run5_frame/frame_0325.pdb
./V135A/Run5_frame/frame_0326.pdb
./V135A/Run5_frame/frame_0327.pdb
./V135A/Run5_frame/frame_0328.pdb
./V135A/Run5_frame/frame_0329.pdb
./V135A/Run5_frame/frame_0330.pdb
./V135A/Run5_frame/frame_0331.pdb
./V135A/Run5_f

./V135A/Run5_frame/frame_0544.pdb
./V135A/Run5_frame/frame_0545.pdb
./V135A/Run5_frame/frame_0546.pdb
./V135A/Run5_frame/frame_0547.pdb
./V135A/Run5_frame/frame_0548.pdb
./V135A/Run5_frame/frame_0549.pdb
./V135A/Run5_frame/frame_0550.pdb
./V135A/Run5_frame/frame_0551.pdb
./V135A/Run5_frame/frame_0552.pdb
./V135A/Run5_frame/frame_0553.pdb
./V135A/Run5_frame/frame_0554.pdb
./V135A/Run5_frame/frame_0555.pdb
./V135A/Run5_frame/frame_0556.pdb
./V135A/Run5_frame/frame_0557.pdb
./V135A/Run5_frame/frame_0558.pdb
./V135A/Run5_frame/frame_0559.pdb
./V135A/Run5_frame/frame_0560.pdb
./V135A/Run5_frame/frame_0561.pdb
./V135A/Run5_frame/frame_0562.pdb
./V135A/Run5_frame/frame_0563.pdb
./V135A/Run5_frame/frame_0564.pdb
./V135A/Run5_frame/frame_0565.pdb
./V135A/Run5_frame/frame_0566.pdb
./V135A/Run5_frame/frame_0567.pdb
./V135A/Run5_frame/frame_0568.pdb
./V135A/Run5_frame/frame_0569.pdb
./V135A/Run5_frame/frame_0570.pdb
./V135A/Run5_frame/frame_0571.pdb
./V135A/Run5_frame/frame_0572.pdb
./V135A/Run5_f

./V135A/Run5_frame/frame_0785.pdb
./V135A/Run5_frame/frame_0786.pdb
./V135A/Run5_frame/frame_0787.pdb
./V135A/Run5_frame/frame_0788.pdb
./V135A/Run5_frame/frame_0789.pdb
./V135A/Run5_frame/frame_0790.pdb
./V135A/Run5_frame/frame_0791.pdb
./V135A/Run5_frame/frame_0792.pdb
./V135A/Run5_frame/frame_0793.pdb
./V135A/Run5_frame/frame_0794.pdb
./V135A/Run5_frame/frame_0795.pdb
./V135A/Run5_frame/frame_0796.pdb
./V135A/Run5_frame/frame_0797.pdb
./V135A/Run5_frame/frame_0798.pdb
./V135A/Run5_frame/frame_0799.pdb
./V135A/Run5_frame/frame_0800.pdb
./V135A/Run5_frame/frame_0801.pdb
./V135A/Run5_frame/frame_0802.pdb
./V135A/Run5_frame/frame_0803.pdb
./V135A/Run5_frame/frame_0804.pdb
./V135A/Run5_frame/frame_0805.pdb
./V135A/Run5_frame/frame_0806.pdb
./V135A/Run5_frame/frame_0807.pdb
./V135A/Run5_frame/frame_0808.pdb
./V135A/Run5_frame/frame_0809.pdb
./V135A/Run5_frame/frame_0810.pdb
./V135A/Run5_frame/frame_0811.pdb
./V135A/Run5_frame/frame_0812.pdb
./V135A/Run5_frame/frame_0813.pdb
./V135A/Run5_f

In [15]:
data = Mercury_SASA_ext

# Prepare DataFrame columns and data
columns = []
data_lists = []

# Process data
for mutation in data:
    for run in data[mutation]:
        for dist_key, chains in data[mutation][run].items():
            for chain_id, values in chains.items():
                col_name = f"{mutation}_{run}_{dist_key}_{chain_id}"
                columns.append(col_name)
                data_lists.append(values)  # Append the whole list, not extend

titles = columns

data = array(data_lists).T.tolist()

In [17]:
import csv
# File path to write
file_path = 'mercury_SASA_all.txt'

# Writing to a file
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')  # Use tab as delimiter
    writer.writerow(titles)  # Write the title row
    writer.writerows(data)  # Write the data rows

In [20]:
len(data[0])

300